In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [162]:
train=pd.read_csv(r"C:\Users\admin\Desktop\DS Python\Logistics regression,clustering, NLP\Titanic-logistic regression\train.csv")
test=pd.read_csv(r"C:\Users\admin\Desktop\DS Python\Logistics regression,clustering, NLP\Titanic-logistic regression\test.csv")

In [14]:
train.shape,test.shape

((891, 12), (418, 11))

In [16]:
train.columns,test.columns

(Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
       dtype='object'),
 Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
        'Ticket', 'Fare', 'Cabin', 'Embarked'],
       dtype='object'))

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [17]:
train.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [36]:
#train.isna().sum() # Age, Cabin, Embarked
#test.isna().sum() # Age, Cabin

In [18]:
test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [138]:
# Transforming sex categorical variable to numeric
def map_sex(data):
    mapped_sex={'male':0,'female':1}
    data['Sex'].fillna('male',inplace=True)
    data['Sex']= data['Sex'].map(mapped_sex)
    #data['Sex'].astype(int)

In [124]:
## Create new features FamilySize, Is_Alone as a combination of SibSp and Parch
def map_family(data):
    data['Family_size']=data['SibSp']+data['Parch']
    data['Family_size']=data['Family_size'].astype(int)
    data['Is_Alone']=0
    data.loc[data['Family_size'] >0,'Is_Alone']=0

In [25]:
# Mapping Fare
def map_fare(data):
    data.loc[data['Fare'] <=7.91,'Fare'] =0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare'] =3
    data['Fare']=data['Fare'].astype(int)

In [150]:
#Mapping cabin
def map_cabin(data):
    data['Cabin'].fillna(0,inplace=True)
    data['Has_cabin']=data['Cabin'].apply(lambda x: 1 if x !=0 else 0)

In [146]:
#Mapping Embarked
def map_embarked(data):
    mapping_embark={'S': 0, 'C': 1, 'Q': 2}
       #data['Embarked']=data['Embarked'].fillna('S')
    data['Embarked']=data['Embarked'].map(mapping_embark)

In [33]:
#Mapping age
def map_age(data):
    data['Age'].fillna(0,inplace=True)
    data.loc[ data['Age'] <= 16, 'Age'] = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[ data['Age'] > 64, 'Age'] = 4 ;

In [134]:
# Define function to extract titles from passenger names
def get_title(data):
    title=[]
    name=data['Name']
    name=name.str.split('.')
    for i in name:
        title.append(i[0].split(',')[1].strip())
    data['Title']=title
    
    data['Title'] = data['Title'].replace(['Don', 'Rev', 'Dr', 'Major', 'Lady', 'Sir', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], 'Rare')

    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
     

In [111]:
#Mapping title extracted from the Names
def map_title(data):
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    data['Title'] = data['Title'].map(title_mapping)
    data['Title'] = data['Title'].fillna(0)


In [112]:
# Feature engineering master
def feature_engineering(dataset):
    map_age(dataset)
    map_cabin(dataset)
    map_embarked(dataset)
    map_family(dataset)
    map_fare(dataset)
    map_sex(dataset)
    get_title(dataset)
    map_title(dataset) 

In [163]:
train['Embarked'] = train['Embarked'].fillna('S')
feature_engineering(train)

In [164]:
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
feature_engineering(test)

In [165]:
features_to_be_dropped = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp','Parch']
train=train.drop(features_to_be_dropped,axis=1)
test=test.drop(features_to_be_dropped,axis=1)

In [166]:
train.head(5)

,Survived,Pclass,Sex,Age,Fare,Embarked,Has_cabin,Family_size,Is_Alone,Title
0,0,3,0,1.0,0,0,0,1,0,1
1,1,1,1,2.0,3,1,1,1,0,3
2,1,3,1,1.0,1,0,0,0,0,2
3,1,1,1,2.0,3,0,1,1,0,3
4,0,3,0,2.0,1,0,0,0,0,1


In [167]:
test.head(5)

,Pclass,Sex,Age,Fare,Embarked,Has_cabin,Family_size,Is_Alone,Title
0,3,0,2.0,0,2,0,0,0,1.0
1,3,1,2.0,0,0,0,1,0,3.0
2,2,0,3.0,1,2,0,0,0,1.0
3,3,0,1.0,1,0,0,0,0,1.0
4,3,1,1.0,1,0,0,2,0,3.0


In [169]:
# Create a classification model for the titanic dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

In [170]:
x=train.drop('Survived',axis=1)
y=train['Survived']
x_train,x_test,y_train, y_test=train_test_split(x,y,test_size=0.3)

logmod=LogisticRegression()
logmod.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [172]:
y_pred=logmod.predict(x_test)
print("Accuracy score is {0:.02f}".format(accuracy_score(y_test,y_pred)))

Accuracy score is 0.83


In [173]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86       164
           1       0.79      0.77      0.78       104

   micro avg       0.83      0.83      0.83       268
   macro avg       0.82      0.82      0.82       268
weighted avg       0.83      0.83      0.83       268



In [174]:
roc_auc_score(y_test,y_pred)

0.8205909943714822